<a href="https://colab.research.google.com/github/PremAround-AI/AI-ERDS/blob/main/AI_ERDS_Final_Project_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ModuleNotFoundError: No module named 'cv2'

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#This code already made sequence folder in different file, so doesn't run the file here.

# Configuration
video_base_dir = "/content/drive/MyDrive/Project_DataSet" # Base directory containing class folders
output_dir = "drive/MyDrive/sequences"
os.makedirs(output_dir, exist_ok=True)

SEQ_LEN = 15
IMG_SIZE = 64
STRIDE = 2
AUGMENTATIONS_PER_CLIP = 2

# Augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=10, p=0.3)
])

def augment_clip(clip):
    """Applies defined augmentations to a list of frames."""
    # Albumentations expects a single image, so we apply it frame by frame
    return [transform(image=frame)["image"] for frame in clip]

print("Starting video processing and augmentation...")

# Check if the base directory exists
if not os.path.isdir(video_base_dir):
    print(f"Error: Directory '{video_base_dir}' not found.")
    print("Please verify the path to your dataset in Google Drive.")
else:
    # Get all class folders and sort them to ensure consistent selection
    all_class_folders = sorted([d for d in os.listdir(video_base_dir) if os.path.isdir(os.path.join(video_base_dir, d))])

    # Use only the first 5 folders (classes)
    class_folders_to_process = all_class_folders[:5]

    print(f"Processing a total of {len(class_folders_to_process)} classes.")

    for label_index, label in enumerate(class_folders_to_process):
        class_path = os.path.join(video_base_dir, label)
        print(f"\nProcessing class {label_index + 1}/{len(class_folders_to_process)}: {label}")

        # Get all video files in the current class folder and sort them
        all_video_files = sorted([f for f in os.listdir(class_path) if f.endswith(".mp4") or f.endswith(".avi")]) # Added .avi just in case

        # Limit to first 40 videos per class (if available)
        video_files_to_process = all_video_files[:40]

        if not video_files_to_process:
            print(f"  No video files found in class: {label}. Skipping.")
            continue

        print(f"  Processing {len(video_files_to_process)} videos from class {label}.")

        for video_index, video_file in enumerate(video_files_to_process):
            video_path = os.path.join(class_path, video_file)
            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                print(f"  Error: Could not open video file {video_path}. Skipping.")
                continue

            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                # Resize frame to IMG_SIZE
                frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
                frames.append(frame)

            cap.release()

            if not frames:
                print(f"  Warning: No frames read from video {video_file}. Skipping clip generation for this video.")
                continue

            # Generate clips and save
            clip_count = 0
            for j in range(0, len(frames) - SEQ_LEN + 1, STRIDE):
                clip = frames[j:j + SEQ_LEN]

                # Ensure the clip has the required sequence length
                if len(clip) != SEQ_LEN:
                    continue # Skip if the last clip is shorter than SEQ_LEN

                # Save original clip
                # The naming convention will be {class_name}_{video_index_in_class}_{clip_start_frame_index}.npy
                clip_filename = f"{label}_{video_index:03d}_{j:03d}.npy"
                np.save(os.path.join(output_dir, clip_filename), np.array(clip))
                clip_count += 1

                # Augment clip and save augmented versions
                for k in range(AUGMENTATIONS_PER_CLIP):
                    aug_clip = augment_clip(clip)
                    aug_clip_filename = f"{label}_{video_index:03d}_{j:03d}_aug{k}.npy"
                    np.save(os.path.join(output_dir, aug_clip_filename), np.array(aug_clip))

            print(f"    Processed video {video_index + 1}/{len(video_files_to_process)} ({video_file}) - Generated {clip_count} clips.")

    print("\nAll specified sequences and augmentations saved to 'sequences/' directory.")

In [ ]:
!pip install tensorflow
import tensorflow
print(tensorflow.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 119.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
2.19.0


In [ ]:
# generate_metadata.py
import os
import numpy as np
import json

SEQUENCE_DIR = "drive/MyDrive/sequences"
SEQ_LEN = 15
IMG_SIZE = 64
metadata = []

print("Generating metadata cache...")

for file in os.listdir(SEQUENCE_DIR):
    if file.endswith(".npy"):
        parts = file.split("_")
        if len(parts) < 2:
            continue
        label = parts[0]
        path = os.path.join(SEQUENCE_DIR, file)
        try:
            shape = np.load(path, mmap_mode='r').shape
            if shape == (SEQ_LEN, IMG_SIZE, IMG_SIZE, 3):
                metadata.append({"file": path, "label": label})
        except:
            continue

with open("sequence_metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

print(f"Saved metadata for {len(metadata)} valid files.")

Generating metadata cache...
Saved metadata for 26886 valid files.


In [ ]:
import os
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import TimeDistributed, GlobalAveragePooling2D, LSTM, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence

# Configuration
SEQUENCE_DIR = "drive/MyDrive/sequences"
SEQ_LEN = 15
IMG_SIZE = 64
BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 0.001
MODEL_SAVE_PATH = "sign_model.h5"
LABEL_MAP_PATH = "label_map.json"
METADATA_PATH = "sequence_metadata.json"

# --- Step 1: Load metadata ---
print("Loading metadata from cache...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

X_files = [entry["file"] for entry in metadata]
y_labels = [entry["label"] for entry in metadata]

print(f"Found {len(X_files)} valid sequence files.")

# --- Step 2: Encode labels ---
le = LabelEncoder()
y_encoded = le.fit_transform(y_labels)

label_map = {str(idx): str(cls) for idx, cls in enumerate(le.classes_)}
with open(LABEL_MAP_PATH, "w") as f:
    json.dump(label_map, f, indent=4)
print(f"Label map saved: {label_map}")

# --- Step 3: Train-test split ---
min_samples_per_class = min(np.bincount(y_encoded))
if min_samples_per_class < 2:
    print("Not enough samples per class for stratified split. Using basic split.")
    X_train_files, X_test_files, y_train, y_test = train_test_split(
        X_files, y_encoded, test_size=0.2, random_state=42)
else:
    X_train_files, X_test_files, y_train, y_test = train_test_split(
        X_files, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)
    print("Stratified train-test split applied.")

print(f"Train: {len(X_train_files)} | Test: {len(X_test_files)}")

# --- Step 4: Data generator ---
class SequenceDataGenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size, shuffle=True):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.file_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_files = [self.file_paths[i] for i in batch_indices]
        batch_labels = [self.labels[i] for i in batch_indices]

        X = np.array([np.load(f).astype('float32') / 255.0 for f in batch_files])
        y = np.array(batch_labels)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

train_gen = SequenceDataGenerator(X_train_files, y_train, BATCH_SIZE)
val_gen = SequenceDataGenerator(X_test_files, y_test, BATCH_SIZE, shuffle=False)

print(f"Train batches per epoch: {len(train_gen)}")
print(f"Validation batches per epoch: {len(val_gen)}")

# --- Step 5: Build the model ---
def build_model(seq_len, img_size, num_classes):
    base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))
    base_model.trainable = False

    model = Sequential([
        TimeDistributed(base_model, input_shape=(seq_len, img_size, img_size, 3)),
        TimeDistributed(GlobalAveragePooling2D()),
        LSTM(128, return_sequences=False),
        Dropout(0.5),
        Dense(64, activation="relu"),
        Dropout(0.3),
        Dense(num_classes, activation="softmax")
    ])
    return model

num_classes = len(le.classes_)
model = build_model(SEQ_LEN, IMG_SIZE, num_classes)
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

# --- Step 6: Train the model ---
print("Training the model...")

checkpoint = ModelCheckpoint(
    MODEL_SAVE_PATH, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'
)
early_stopping = EarlyStopping(
    monitor='val_loss', patience=10, min_delta=1e-4, verbose=1, mode='min', restore_best_weights=True
)

callbacks_list = [checkpoint, early_stopping]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks_list,
    verbose=2
)

# --- Step 7: Evaluate the model ---
print("Evaluating on the test set...")
best_model = load_model(MODEL_SAVE_PATH)
loss, acc = best_model.evaluate(val_gen, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {acc:.4f}")

Loading metadata from cache...
Found 26886 valid sequence files.
Label map saved: {'0': 'fire', '1': 'road', '2': 'voilence'}
Stratified train-test split applied.
Train: 21508 | Test: 5378
Train batches per epoch: 2689
Validation batches per epoch: 673


/tmp/ipython-input-2-154406971.py:90: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_2              │ (None, 15, 2, 2, 1280) │     2,257,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 15, 1280)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       721,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,987,843 (11.40 MB)

...

Training the model...
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to sign_model.h5


2689/2689 - 1615s - 601ms/step - accuracy: 0.9897 - loss: 0.0297 - val_accuracy: 1.0000 - val_loss: 6.5430e-05
Epoch 2/5

Epoch 2: val_accuracy did not improve from 1.00000
2689/2689 - 1557s - 579ms/step - accuracy: 0.9964 - loss: 0.0146 - val_accuracy: 1.0000 - val_loss: 4.6786e-06
Epoch 3/5

Epoch 3: val_accuracy did not improve from 1.00000
2689/2689 - 1553s - 577ms/step - accuracy: 1.0000 - loss: 2.4852e-04 - val_accuracy: 1.0000 - val_loss: 1.6543e-07
Epoch 4/5

Epoch 4: val_accuracy did not improve from 1.00000
2689/2689 - 1535s - 571ms/step - accuracy: 1.0000 - loss: 4.5391e-05 - val_accuracy: 1.0000 - val_loss: 4.7318e-07
Epoch 5/5

Epoch 5: val_accuracy did not improve from 1.00000
2689/2689 - 1562s - 581ms/step - accuracy: 0.9981 - loss: 0.0078 - val_accuracy: 1.0000 - val_loss: 2.9719e-07
Restoring model weights from the end of the best epoch: 1.
Evaluating on the test set...


Test Loss: 0.0001, Test Accuracy: 1.0000


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import json
from sklearn.preprocessing import LabelEncoder


VIDEO_PATH = "/content/drive/MyDrive/Project_DataSet/fire/WhatsApp Video 2025-06-21 at 01.10.32_b91ff951.mp4"
SEQ_LEN = 15
IMG_SIZE = 64
MODEL_PATH = "sign_model.h5"
LABEL_MAP_PATH = "label_map.json"


model = load_model(MODEL_PATH)

with open(LABEL_MAP_PATH, "r") as f:
    label_map = json.load(f)

inv_label_map = {int(k): v for k, v in label_map.items()}


def load_video_frames(video_path, seq_len=15, img_size=64):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while len(frames) < seq_len:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_size, img_size))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()

    if len(frames) < seq_len:
        print(f"Video has only {len(frames)} frames. Padding with last frame.")
        while len(frames) < seq_len:
            frames.append(frames[-1])  # pad with last frame

    frames = np.array(frames).astype("float32") / 255.0
    return np.expand_dims(frames, axis=0)  # shape: (1, seq_len, img_size, img_size, 3)


sample_sequence = load_video_frames(VIDEO_PATH, SEQ_LEN, IMG_SIZE)
prediction = model.predict(sample_sequence)
predicted_class = np.argmax(prediction, axis=1)[0]
predicted_label = inv_label_map[predicted_class]

print(f"Predicted Label: {predicted_label}")

ModuleNotFoundError: No module named 'cv2'

In [ ]:
!pip install twilio
!pip install open-cv python

ERROR: Could not find a version that satisfies the requirement open-cv (from versions: none)
ERROR: No matching distribution found for open-cv


In [ ]:
# Twilio credentials
ACCOUNT_SID = "AC58fa4f42a75cc9f2bdc4bf738d495beb"
AUTH_TOKEN = "2409004567c530a4355b2254d940938e"
FROM_PHONE = "+16163445781"
TO_PHONE = "+917989275641"

In [ ]:
!pip install twilio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.2 MB/s eta 0:00:00


In [ ]:
from twilio.rest import Client

def send_sms_alert(message_body):
    try:
        client = Client(ACCOUNT_SID, AUTH_TOKEN)
        message = client.messages.create(
            body=message_body,
            from_=FROM_PHONE,
            to=TO_PHONE
        )
        print(f"✅ SMS sent: SID {message.sid}")    #online copied icons use to look attractive
    except Exception as e:
        print(f"❌ SMS failed: {e}")

In [ ]:
if predicted_label == "fire":
    send_sms_alert("🔥 Fire Detected! Immediate attention needed.")
elif predicted_label=="road_accident":
    send_sms_alert("Road Accident Detected! Immediate attention needed.")
elif predicted_label=="voilence":
    send_sms_alert("Voilence Detected! Immediate attention needed.")
else:
    print("No alert needed.")

NameError: name 'predicted_label' is not defined